In [ ]:
# Task 2.1. Frequency components of a synthetic time-series signal
#2.1.1 Draw a line plot of the series

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from random import gauss
from random import seed
from pandas import Series
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller

# Parameters
total_duration = 5  # Total duration of the signal in seconds
sampling_rate = 200  # Sampling rate in Hz

# Time array for the entire signal
t = np.linspace(0, total_duration, total_duration * sampling_rate, endpoint=False)

frequencies = [10, 20, 30, 40, 50]  # Frequencies in Hz for each second
signal = np.zeros_like(t)

for i, freq in enumerate(frequencies, 1):
    mask = (t >= i - 1) & (t < i)  # Mask for each second
    signal[mask] = np.sin(2 * np.pi * freq * t[mask])
    

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(t, signal)
plt.title('Sine Wave with combined frequencies')
plt.xlabel('Time (seconds)')
plt.ylabel('Amplitude')
plt.savefig('combined_sines_lineplot.png')
plt.grid(True)
plt.show()


In [ ]:
#2.1.2 Draw the power spectrum
# Compute the FFT
fft_result = np.fft.fft(signal)
fft_freq = np.fft.fftfreq(len(signal), 1/sampling_rate)

# Compute the power spectrum
power_spectrum = np.abs(fft_result)**2


# Plot the power spectrum of the entire series
plt.figure(figsize=(10, 6))
plt.plot(fft_freq[:len(fft_freq)//2], power_spectrum[:len(fft_result)//2])
plt.title("Power Spectrum of Entire Series")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.savefig('sines_powerspectrum.png')
plt.grid(True)
plt.show()

In [ ]:
#2.1.3 Draw the spectogram of the series
# Create spectrogram of the entire series
plt.figure(figsize=(10, 6))
plt.specgram(signal, Fs=sampling_rate, NFFT=256, noverlap=128)
plt.title("Spectrogram of Entire Series")
plt.xlabel("Time (s)")
plt.ylabel("Frequency (Hz)")
plt.colorbar(label="Intensity (dB)")
plt.savefig('sines_spectrogram.png')
plt.show()

In [ ]:
#Draw and compare the ACF and PACF graphs of the first one-second (frequency10Hz) and the second one-second series (frequency 20Hz), with lags up to 50.


# Parameters
duration = 1  # Duration of each series in seconds
sampling_rate = 200  # Hz

# Generate time array for each series
num_samples = duration * sampling_rate
time = np.linspace(0, duration, num_samples)

# Generate sine wave signals for frequency 10Hz and 20Hz
freq_10 = 10
freq_20 = 20

signal_10 = np.sin(2 * np.pi * freq_10 * time)
signal_20 = np.sin(2 * np.pi * freq_20 * time)


# Plot the two series
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(time, signal_10)
plt.title("Signal for Frequency 10Hz")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.subplot(2, 1, 2)
plt.plot(time, signal_20)
plt.title("Signal for Frequency 20Hz")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")


plt.tight_layout()
plt.show()
# Calculate ACF and PACF for both series
lags = 50

# For Frequency 10Hz
plt.figure(figsize=(10, 6))
plot_acf(signal_10, lags=lags, title="ACF - Frequency 10Hz")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.savefig("ACF10HZ.png")
plt.show()

plt.figure(figsize=(10, 6))
plot_pacf(signal_10, lags=lags, title="PACF - Frequency 10Hz")
plt.xlabel("Lag")
plt.ylabel("Partial Autocorrelation")
plt.savefig("PACF10HZ.png")
plt.show()

# For Frequency 20Hz
plt.figure(figsize=(10, 6))
plot_acf(signal_20, lags=lags, title="ACF - Frequency 20Hz")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.savefig("ACF20HZ.png")
plt.show()

plt.figure(figsize=(10, 6))
plot_pacf(signal_20, lags=lags, title="PACF - Frequency 20Hz")
plt.xlabel("Lag")
plt.ylabel("Partial Autocorrelation")
plt.savefig("PACF20HZ.png")
plt.show()

In [ ]:
#Task 2.2. Statistical features and discovery of event-related potential
#2.2.1 Visualize the response, i.e., ERP of the EEG, in the two conditions, A and B.
from scipy.io import loadmat

 # Import function to read data.
data = loadmat(r"C:\Users\irene\Desktop\Universidad\Master\KTH\Segundo\Period 4\Embedded Intelligence\Lab 1\02_EEG-1.mat")
data.keys()
EEGa = data['EEGa']
EEGb = data['EEGb']
t = data['t'][0]

# Calculate ERP for Condition A and Condition B
erp_A = np.mean(EEGa, axis=0)
erp_B = np.mean(EEGb, axis=0)
erp_A.shape
# Plot ERP for Condition A
plt.figure(figsize=(10, 6))
plt.plot(t, erp_A)
plt.title('Event-Related Potential (ERP) - Condition A')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.savefig("ECGA.png")
plt.show()

# Plot ERP for Condition B
plt.figure(figsize=(10, 6))
plt.plot(t, erp_B)
plt.title('Event-Related Potential (ERP) - Condition B')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.savefig("ECGB.png")
plt.show()

In [ ]:
# 2.2.2 Find the brain activity frequency in the data of condition A
from statsmodels.tsa.seasonal import seasonal_decompose
# Apply seasonal decomposition
result = seasonal_decompose(erp_A, model='additive', period=12)

# Get the trend, seasonal, and residual components
trend = result.trend
seasonal = result.seasonal
residual = result.resid

# Plot the components
plt.figure(figsize=(12, 8))

plt.subplot(411)
plt.plot(erp_A, label='Original', color='blue')
plt.legend(loc='upper left')

plt.subplot(412)
plt.plot(trend, label='Trend', color='red')
plt.legend(loc='upper left')

plt.subplot(413)
plt.plot(seasonal, label='Seasonal', color='green')
plt.legend(loc='upper left')

plt.subplot(414)
plt.plot(residual, label='Residual', color='purple')
plt.legend(loc='upper left')

plt.tight_layout()
plt.savefig('Erpa_seasonaldecomposition.png')
plt.show()

In [ ]:
# Task 2.3. Features of observed rhythms in EEG
# 2.3.1  For the EEG data set, draw the line plot, histogram, density plot, box plot, lag-1, plot, ACF and PACF graphs (lags up to 50).
from scipy.io import loadmat

# Import function to read data.
data = loadmat("C:/Users/irene/Desktop/Universidad/Master/KTH/Segundo/Period 4/Embedded Intelligence/Lab 1/03_EEG-1.mat")
# Extract the time and corresponding y values
t = data['t'].flatten()  # Assuming 't' is a 1D array
y = data['EEG'].flatten()  # Assuming 'y' is a 1D array

#Plots
#Line plot
plt.figure(figsize=(10, 5))
plt.plot(t,y)
plt.title("EEG data")
plt.xlabel("Time")
plt.ylabel("Value")
plt.savefig("ECGlineplot.png")
plt.show()
#Histogram
plt.figure(figsize=(10, 5))
plt.hist(y, bins=30, density=True, alpha=0.75)
plt.title("Histogram of EEG data")
plt.xlabel("Value")
plt.ylabel("Frequency")
plt.savefig("ECGhistogram.png")
plt.show()
#Density plot
plt.figure(figsize=(10, 5))
sns.kdeplot(y, fill=True)
plt.title("Density Plot of EEG data")
plt.xlabel("Value")
plt.ylabel("Density")
plt.savefig("ECGdensity.png")
plt.show()
#Box plot
plt.figure(figsize=(8, 6))
plt.boxplot(y)
plt.title("Box Plot of EEG data")
plt.ylabel("Value")
plt.savefig("ECGbox.png")
plt.show()
#Lag - 1 Plot
lag1 = np.roll(y, 1)
plt.figure(figsize=(8, 6))
plt.scatter(y, lag1)
plt.title("Lag-1 Plot of EEG data")
plt.xlabel("White Noise(t)")
plt.ylabel("White Noise(t-1)")
plt.savefig("ECGlag.png")
plt.show()
# Plot ACF using pandas.plotting.autocorrelation_plot
plt.figure(figsize=(12, 6))
autocorrelation_plot(y)
plt.title("Autocorrelation Function (ACF) of EEG data")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.savefig("ECGacf.png")
plt.show()
plt.figure(figsize=(12, 8))
plot_pacf(y, lags=40, alpha=0.05)
plt.title("PACF of EEG data")
plt.xlabel("Lag")
plt.ylabel("Partial Autocorrelation")
plt.savefig("ECGlpacf.png")
plt.show()

In [ ]:
#2.3.2 Show the statistical characteristics of the EEG data, such as mean, variance, standard deviation.
# Compute the mean, variance, and standard deviation
mean_EEG = np.mean(y)
variance_EEG = np.var(y)
std_dev_EEG = np.std(y)

# Print the results
print("Mean of EEG Data:", mean_EEG)
print("Variance of EEG Data:", variance_EEG)
print("Standard Deviation of EEG Data:", std_dev_EEG)

In [ ]:
#2.3.3  Compute the auto-covariance of the EEG data. Draw and save the auto-covariance graph. 
# Compute auto-covariance using numpy for positive lags only
auto_covariance = np.correlate(y - np.mean(y), y - np.mean(y), mode='same') / len(y)

# Plot auto-covariance
plt.figure(figsize=(10, 6))
plt.plot(t[0:200], auto_covariance[0:200], label='Auto-covariance')
plt.title("Auto-covariance of EEG Data")
plt.xlabel("Lag")
plt.ylabel("Auto-covariance")
plt.grid(True)
plt.legend()
plt.savefig("auto_covariance_plot.png")  # Save the plot
plt.show()

In [ ]:
#2.3.4  Compute and plot the power-spectrum of the EEG data. Show it in both linear scale and log (dB) scale.
# Sampling rate (assuming data is sampled at a constant rate)
sampling_rate = 1000  # Hz

# Compute FFT of the EEG data
fft_vals = np.fft.fft(y)
freqs = np.fft.fftfreq(len(fft_vals), 1/sampling_rate)

# Compute power spectrum
power_spectrum = np.abs(fft_vals) ** 2
for n in range(len(power_spectrum)):
    if(power_spectrum[n] > 10000):
        print(power_spectrum[n], freqs[n])
        
    
# Plot power spectrum in linear scale
plt.figure(figsize=(10, 6))
plt.plot(freqs, power_spectrum)
plt.title("Power Spectrum of EEG Data (Linear Scale)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power")
plt.grid(True)
plt.savefig("power_spectrum_linear.png")  # Save the plot
plt.show()

# Plot power spectrum in log scale (dB)
power_spectrum_dB = 10 * np.log10(power_spectrum)  # Convert to dB scale

plt.figure(figsize=(10, 6))
plt.plot(freqs, power_spectrum_dB)
plt.title("Power Spectrum of EEG Data (Log Scale - dB)")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Power (dB)")
plt.grid(True)
plt.savefig("power_spectrum_log.png")  # Save the plot
plt.show()

In [ ]:
#Additional question
#Assume a short time-series {1,2,3,4,5,6,7,8,7,6,5,4,3,2,1}. (1) Calculate theauto-covariance and auto-correlations for all valid lags. Do the calculations manually. 
# (2) Write a Python program to validate your calculations.
import numpy as np
import matplotlib.pyplot as plt

# Given Time Series
x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 7, 6, 5, 4, 3, 2, 1])

# Mean
mean = np.mean(x)
variance = np.var(x)
N = 15
sum_covariance = 0
auto_covariance = []
auto_correlation = []
for l in range(14,-1,-1):
    sum_covariance = 0
    for n in range(N - l):
        sum_covariance += (x[n + l] - mean) * (x[n] - mean)

    auto_covariance.append((1 / N) * sum_covariance)
    auto_correlation.append(((1 / N) * sum_covariance)/(variance*variance))

# Plot ACF using pandas.plotting.autocorrelation_plot
plt.figure(figsize=(12, 6))
autocorrelation_plot(x)
plt.title("Autocorrelation Function (ACF) of Random-walk series")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation")
plt.savefig('acf_x.png')
plt.show()



